In [43]:
# re
import re

# data reading / cleaning
from gut_tokenize import read_data, preprocess

# ceiling 
import math 

# random
import random
random.seed(42)

import nltk

In [44]:
directory = "../Gutenberg/relic_test_set/"
titles = []
texts = []
titles, texts = read_data(directory)

In [45]:
def paragraph_tokenize(text):
    text = text.split("\n\n")
    text = [re.sub("\s{2,}", " ", t.replace("\n", "").strip()) for t in text]
    return text